In [1]:
import copy
import itertools
import logging

In [2]:
class LoggerMixin(object):
    logger_level = logging.ERROR

    @property
    def logger(self):
        logger = logging.getLogger(self.__class__.__name__)
        if (logger.hasHandlers()):
            logger.handlers.clear()
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(name)s - %(levelname)s %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.setLevel(self.logger_level)
        return logger


In [3]:
class Matrix(LoggerMixin, object):
    def __init__(self, data):
        n = len(data)  # number of rows
        m = len(data[0])  # number of columns
        self.shape = (n, m)
        self.data = [[float(element) for element in row] for row in data]

    @classmethod
    def zeros(cls, shape):
        n, m = shape
        matrix = cls([
            [0 for k in range(m)]
            for j in range(n)
        ])
        return matrix

    @classmethod
    def identity(cls, n):
        matrix = cls.zeros((n, n))
        for j in range(n): matrix[j][j] = 1.0
        return matrix

    def is_square(self):
        return self.shape[0] == self.shape[1]

    def copy(self):
        return Matrix(copy.deepcopy(self.data))

    def __len__(self):
        return self.n

    def __repr__(self):
        return "Matrix([\n    " \
            + ",\n    ".join(repr(row) for row in self.data) + "\n])"

    def __getitem__(self, item):
        return self.data[item]

    def __add__(self, other):
        result = Matrix.zeros(self.shape)
        for j, k in itertools.product(range(result.shape[0]), range(result.shape[1])):
            try:
                assert self.shape == other.shape
                result[j][k] = self[j][k] + other[j][k]
            except AttributeError:
                result[j][k] = self[j][k] + other
        return result

    def __sub__(self, other):
        result = Matrix.zeros(self.shape)
        for j, k in itertools.product(range(result.shape[0]), range(result.shape[1])):
            try:
                assert self.shape == other.shape
                result[j][k] = self[j][k] - other[j][k]
            except AttributeError:
                result[j][k] = self[j][k] - other
        return result

    def __mul__(self, other):
        result = self.copy()
        for j, k in itertools.product(range(result.shape[0]), range(result.shape[1])):
            try:
                assert self.shape == other.shape
                result[j][k] *= other[j][k]
            except AttributeError:
                result[j][k] *= other
        return result
    
    def __matmul__(self, other):
        # hehehe
        return self.multiply_by(other)

    def add(self, other):
        return self + other

    def multiply_by(self, other):
        assert self.shape[1] == other.shape[0]
        return Matrix([
            [
                float(sum(
                    self.data[self_row][self_column] \
                        * other.data[other_row][other_column]
                    for self_column, other_row in 
                        zip(range(self.shape[1]), range(other.shape[0]))
                ))
                for other_column in range(other.shape[1])
            ]
            for self_row in range(self.shape[0])
        ])

    def transpose(self):
        new = Matrix.zeros((self.shape[1], self.shape[0]))
        for j, k in itertools.product(range(self.shape[0]), range(self.shape[1])):
            new[k][j] = self[j][k]
        return new

    @property
    def T(self):
        return self.transpose()

    def invert(self):
        n = self.shape[0]
        operations_so_far = Matrix.identity(n)
        self_copy = self.copy()

        for k in range(n):
            self.logger.info(f"processing column {k}")
            self.logger.info("self_copy = " + str(self_copy))

            # reorder rows
            self.logger.info("reorder rows")
            row_reordering = \
                list(range(k)) + \
                list(reversed(sorted(
                    range(k, n),
                    key=lambda j: abs(self_copy[j][k])
            )))
            reorder_op = Matrix([Matrix.identity(n).data[j] for j in row_reordering])
            self.logger.debug("reorder_op = " + str(reorder_op))
            operations_so_far = reorder_op.multiply_by(operations_so_far)
            self_copy = reorder_op.multiply_by(self_copy)
            self.logger.info("self_copy = " + str(self_copy))

            # normalize this row
            self.logger.info("normalize this row")
            normalization_op = Matrix.identity(n)
            normalization_op[k][k] = 1.0 / self_copy[k][k]
            self.logger.debug("normalization_op = " + str(normalization_op))
            operations_so_far = normalization_op.multiply_by(operations_so_far)
            self_copy = normalization_op.multiply_by(self_copy)
            self.logger.info("self_copy = " + str(self_copy))

            # eliminate the other rows
            self.logger.info("eliminate the other rows")
            eliminations_op = Matrix.identity(n)
            for j in range(n):
                if j == k:
                    continue
                eliminations_op[j][k] = -1.0 * self_copy[j][k]
            self.logger.debug("eliminations_op = " + str(eliminations_op))
            operations_so_far = eliminations_op.multiply_by(operations_so_far)
            self.logger.debug("operations_so_far = " + str(operations_so_far))
            self_copy = eliminations_op.multiply_by(self_copy)
            self.logger.info("self_copy = " + str(self_copy))

            self.logger.info(f"done with column {k}\n")
            
        return operations_so_far


# Basics

In [4]:
X = Matrix([[1, 2], [3, 4], [5, 6]])
X

Matrix([
    [1.0, 2.0],
    [3.0, 4.0],
    [5.0, 6.0]
])

In [5]:
Y = Matrix([[1, 2, 3], [4, 5, 6]])
Y

Matrix([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])

In [6]:
Matrix.zeros((2, 3))

Matrix([
    [0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0]
])

In [7]:
Matrix.identity(8)

Matrix([
    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
])

# Matrix addition

In [8]:
X = Matrix([
    [1, 3]
])
Y = Matrix([
    [5, 11]
])

In [9]:
X.add(Y)

Matrix([
    [6.0, 14.0]
])

In [10]:
Y.add(X)

Matrix([
    [6.0, 14.0]
])

# Matrix multiplication

In [11]:
Matrix([
    [2]
]).multiply_by(Matrix([
    [3]
]))

Matrix([
    [6.0]
])

In [12]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 1]
]))

Matrix([
    [1.0, 2.0],
    [3.0, 4.0]
])

In [13]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [0, 1],
    [1, 0]
]))

Matrix([
    [2.0, 1.0],
    [4.0, 3.0]
])

In [14]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [2, 0],
    [0, 2]
]))

Matrix([
    [2.0, 4.0],
    [6.0, 8.0]
])

In [15]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 0],
    [0, 2]
]))

Matrix([
    [1.0, 4.0],
    [3.0, 8.0]
])

In [16]:
Matrix([
    [1, 2],
    [3, 4]
]).multiply_by(Matrix([
    [1, 1],
    [0, 1]
]))

Matrix([
    [1.0, 3.0],
    [3.0, 7.0]
])

# Matrix transpose

In [17]:
X = Matrix([
    [1]
])
print(X)
print(X.shape)
X_t = X.transpose()
print(X_t)
print(X_t.shape)


Matrix([
    [1.0]
])
(1, 1)
Matrix([
    [1.0]
])
(1, 1)


In [18]:
X = Matrix([
    [1, 0]
])
print(X)
print(X.shape)
X_t = X.transpose()
print(X_t)
print(X_t.shape)


Matrix([
    [1.0, 0.0]
])
(1, 2)
Matrix([
    [1.0],
    [0.0]
])
(2, 1)


In [19]:
X = Matrix([
    [1, 2, 3],
    [4, 5, 6]
])
print(X)
print(X.shape)
X_t = X.transpose()
print(X_t)
print(X_t.shape)


Matrix([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0]
])
(2, 3)
Matrix([
    [1.0, 4.0],
    [2.0, 5.0],
    [3.0, 6.0]
])
(3, 2)


# Matrix inversion

In [20]:
X = Matrix([
    [1, 0],
    [0, 1]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [21]:
X = Matrix([
    [1, 0],
    [0, 2]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [1.0, 0.0],
    [0.0, 2.0]
])
X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 0.5]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [22]:
X = Matrix([
    [2, 0],
    [0, 1]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [2.0, 0.0],
    [0.0, 1.0]
])
X_inv = Matrix([
    [0.5, 0.0],
    [0.0, 1.0]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [23]:
X = Matrix([
    [1, 0, 0],
    [0, 2, 0],
    [0, 0, 3]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 2.0, 0.0],
    [0.0, 0.0, 3.0]
])
X_inv = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.5, 0.0],
    [0.0, 0.0, 0.3333333333333333]
])
X_inv X = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])


In [24]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

X = Matrix([
    [3.0, 1.0],
    [4.0, 2.0]
])
X_inv = Matrix([
    [1.0, -0.5],
    [-2.0, 1.5]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [25]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X.logger_level = logging.DEBUG

X_inv = X.invert()

print("X =", X)
print("X_inv =", X_inv)
print("X_inv X =", X_inv.multiply_by(X))
print("X X_inv =", X.multiply_by(X_inv))

Matrix - INFO processing column 0
Matrix - INFO self_copy = Matrix([
    [3.0, 1.0],
    [4.0, 2.0]
])
Matrix - INFO reorder rows
Matrix - DEBUG reorder_op = Matrix([
    [0.0, 1.0],
    [1.0, 0.0]
])
Matrix - INFO self_copy = Matrix([
    [4.0, 2.0],
    [3.0, 1.0]
])
Matrix - INFO normalize this row
Matrix - DEBUG normalization_op = Matrix([
    [0.25, 0.0],
    [0.0, 1.0]
])
Matrix - INFO self_copy = Matrix([
    [1.0, 0.5],
    [3.0, 1.0]
])
Matrix - INFO eliminate the other rows
Matrix - DEBUG eliminations_op = Matrix([
    [1.0, 0.0],
    [-3.0, 1.0]
])
Matrix - DEBUG operations_so_far = Matrix([
    [0.0, 0.25],
    [1.0, -0.75]
])
Matrix - INFO self_copy = Matrix([
    [1.0, 0.5],
    [0.0, -0.5]
])
Matrix - INFO done with column 0

Matrix - INFO processing column 1
Matrix - INFO self_copy = Matrix([
    [1.0, 0.5],
    [0.0, -0.5]
])
Matrix - INFO reorder rows
Matrix - DEBUG reorder_op = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
Matrix - INFO self_copy = Matrix([
    [1.0, 0.

X = Matrix([
    [3.0, 1.0],
    [4.0, 2.0]
])
X_inv = Matrix([
    [1.0, -0.5],
    [-2.0, 1.5]
])
X_inv X = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])
X X_inv = Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])


In [26]:
X = Matrix([
    [2, 5, 0],
    [1, 0, 1],
    [1, 2, 0],
])
X_inv = X.invert()

print(X)
print(X_inv)
print(X @ X_inv)
print(X_inv @ X)

Matrix([
    [2.0, 5.0, 0.0],
    [1.0, 0.0, 1.0],
    [1.0, 2.0, 0.0]
])
Matrix([
    [-2.0, 0.0, 5.0],
    [1.0, 0.0, -2.0],
    [2.0, 1.0, -5.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])


In [27]:
X = Matrix([
    [2, 5, 0],
    [1, 0, 1],
    [1, 2, 0],
])
X.logger_level = logging.DEBUG
X_inv = X.invert()

print(X)
print(X_inv)
print(X.multiply_by(X_inv))
print(X_inv.multiply_by(X))

Matrix - INFO processing column 0
Matrix - INFO self_copy = Matrix([
    [2.0, 5.0, 0.0],
    [1.0, 0.0, 1.0],
    [1.0, 2.0, 0.0]
])
Matrix - INFO reorder rows
Matrix - DEBUG reorder_op = Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 0.0]
])
Matrix - INFO self_copy = Matrix([
    [2.0, 5.0, 0.0],
    [1.0, 2.0, 0.0],
    [1.0, 0.0, 1.0]
])
Matrix - INFO normalize this row
Matrix - DEBUG normalization_op = Matrix([
    [0.5, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
Matrix - INFO self_copy = Matrix([
    [1.0, 2.5, 0.0],
    [1.0, 2.0, 0.0],
    [1.0, 0.0, 1.0]
])
Matrix - INFO eliminate the other rows
Matrix - DEBUG eliminations_op = Matrix([
    [1.0, 0.0, 0.0],
    [-1.0, 1.0, 0.0],
    [-1.0, 0.0, 1.0]
])
Matrix - DEBUG operations_so_far = Matrix([
    [0.5, 0.0, 0.0],
    [-0.5, 0.0, 1.0],
    [-0.5, 1.0, 0.0]
])
Matrix - INFO self_copy = Matrix([
    [1.0, 2.5, 0.0],
    [0.0, -0.5, 0.0],
    [0.0, -2.5, 1.0]
])
Matrix - INFO done with column 0

M

Matrix([
    [2.0, 5.0, 0.0],
    [1.0, 0.0, 1.0],
    [1.0, 2.0, 0.0]
])
Matrix([
    [-2.0, 0.0, 5.0],
    [1.0, 0.0, -2.0],
    [2.0, 1.0, -5.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])
Matrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0]
])


### demo cute operations

In [28]:
X = Matrix([
    [3, 1],
    [4, 2]
])
X_inv = X.invert()

In [29]:
X + X_inv

Matrix([
    [4.0, 0.5],
    [2.0, 3.5]
])

In [30]:
X - X_inv

Matrix([
    [2.0, 1.5],
    [6.0, 0.5]
])

In [31]:
X * X_inv  # element-wise

Matrix([
    [3.0, -0.5],
    [-8.0, 3.0]
])

In [32]:
X @ X_inv  # matrix-wise

Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])

In [33]:
X_inv @ X

Matrix([
    [1.0, 0.0],
    [0.0, 1.0]
])

# Linear regression

In [34]:
import numpy as np  # only for testing, silly

np.set_printoptions(suppress=True, precision=6)

In [35]:
N = 1000
sigma = 10.0
beta = np.array([10.0, -7.0])
p = len(beta)

In [36]:
# just using Matrix objects here...

rng = np.random.RandomState(seed=0)

beta_ = Matrix([
    [beta[p_]]
    for p_ in range(p)
])
X = Matrix([
    [x[p_] for p_ in range(p)]
    for x in rng.uniform(size=N*p).reshape((N, p))
])
error = Matrix([
    [element] for element in rng.normal(size=N, scale=sigma)
])

In [37]:
X_beta = X @ beta_
y = X_beta + error

In [38]:
print("X =", X[:1])
print("beta_ =", beta_)
print("error =", error[:1])
print("y =", y[:1])

X = [[0.5488135039273248, 0.7151893663724195]]
beta_ = Matrix([
    [10.0],
    [-7.0]
])
error = [[-1.0497970101895355]]
y = [[-0.5679875355232247]]


In [39]:
beta_hat = (X.transpose().multiply_by(X)).invert().multiply_by(X.transpose()).multiply_by(y)
beta_hat

Matrix([
    [9.181662648861964],
    [-7.119123491913385]
])

In [40]:
beta_hat = (X.T @ X).invert() @ X.T @ y
beta_hat

Matrix([
    [9.181662648861964],
    [-7.119123491913385]
])

In [41]:
# which is pretty close to the true beta
beta

array([10., -7.])

In [42]:
# check...
import sklearn as skl
from sklearn.linear_model import LinearRegression

In [43]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X.data, y.data)
lr.coef_

array([[ 9.181663, -7.119123]])

In [44]:
import statsmodels.api as sm
import numpy as np

In [45]:
X_np = np.array(X.data)
X_np[:10], X_np.shape

(array([[0.548814, 0.715189],
        [0.602763, 0.544883],
        [0.423655, 0.645894],
        [0.437587, 0.891773],
        [0.963663, 0.383442],
        [0.791725, 0.528895],
        [0.568045, 0.925597],
        [0.071036, 0.087129],
        [0.020218, 0.83262 ],
        [0.778157, 0.870012]]), (1000, 2))

In [46]:
y_np = np.array(y.data)
y_np[:10], y_np.shape

(array([[ -0.567988],
        [ 15.887601],
        [-16.838151],
        [ -0.330094],
        [ -8.892199],
        [ 12.659529],
        [-12.927409],
        [  2.938151],
        [ -8.448114],
        [ -9.890549]]), (1000, 1))

In [47]:
model = sm.OLS(y_np, X_np)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.117
Model:                            OLS   Adj. R-squared (uncentered):              0.115
Method:                 Least Squares   F-statistic:                              66.05
Date:                Thu, 10 Oct 2019   Prob (F-statistic):                    1.15e-27
Time:                        17:57:05   Log-Likelihood:                         -3693.3
No. Observations:                1000   AIC:                                      7391.
Df Residuals:                     998   BIC:                                      7400.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             9.1817      0.800     11.483      0.000       7.613      10.751
x2            -7.1191      0.797     -8.934      0.000      -8.683      -5.555
==============================================================================
Omnibus:                        3.203   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.202   Jarque-Bera (JB):                2.706
Skew:                          -0.003   Prob(JB):                        0.258
Kurtosis:                       2.745   Cond. No.                         2.65
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

which is pretty close to the true beta!

In [48]:
print("coefficients:")
for p_ in range(p):
    beta_hat_p = beta_hat[p_][0]
    print(f"beta_hat[{p_}]: {beta_hat_p:10.4f}")


coefficients:
beta_hat[0]:     9.1817
beta_hat[1]:    -7.1191


## What about standard errors of the beta estimates?

In [49]:
y_hat = X @ beta_hat
y_hat[:10]

[[-0.05250096910796298],
 [1.6552793077572017],
 [-0.7083445065202598],
 [-2.3308639660514645],
 [6.118258849743972],
 [3.504083962365871],
 [-1.3738432422199924],
 [0.031944877938723026],
 [-5.7418849976759345],
 [0.9510488523220637]]

In [50]:
residual = y - y_hat
residual[:10]

[[-0.5154865664152617],
 [14.232321996582545],
 [-16.129806674654066],
 [2.000769681360495],
 [-15.010457508023478],
 [9.155445046236306],
 [-11.553565776979559],
 [2.9062061499628165],
 [-2.7062287136607255],
 [-10.841598232391581]]

In [51]:
residual_sum_of_squares = residual.T @ residual
residual_sum_of_squares[:10]

[[94507.18524021353]]

In [52]:
(X.shape[0] - X.shape[1])

998

In [53]:
sigma_hat_squared = residual_sum_of_squares.data[0][0] / (X.shape[0] - X.shape[1])
sigma_hat_squared

94.69657839700754

In [54]:
sigma_hat = sigma_hat_squared ** 0.5
sigma_hat

9.73121669664218

In [55]:
# what was sigma again?
sigma

10.0

In [56]:
var_beta_hat = (X.T @ X).invert() * sigma_hat_squared

In [57]:
var_beta_hat

Matrix([
    [0.6392877363450508, -0.47837227675521693],
    [-0.47837227675521693, 0.635052037184252]
])

In [58]:
beta_hat

Matrix([
    [9.181662648861964],
    [-7.119123491913385]
])

In [59]:
print("coefficients and standard errors of the coefficients:")
for p_ in range(p):
    beta_p = beta[p_]
    beta_hat_p = beta_hat[p_][0]
    sigma_hat_p = var_beta_hat[p_][p_] ** 0.5
    print(f"beta[{p_}]: {beta_p:10.4f} \t beta_hat[{p_}]: {beta_hat_p:10.4f} \t std err: {sigma_hat_p:10.4f}")


coefficients and standard errors of the coefficients:
beta[0]:    10.0000 	 beta_hat[0]:     9.1817 	 std err:     0.7996
beta[1]:    -7.0000 	 beta_hat[1]:    -7.1191 	 std err:     0.7969


and these match the results in the `statsmodels` summary:

In [60]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.117
Model:                            OLS   Adj. R-squared (uncentered):              0.115
Method:                 Least Squares   F-statistic:                              66.05
Date:                Thu, 10 Oct 2019   Prob (F-statistic):                    1.15e-27
Time:                        17:57:06   Log-Likelihood:                         -3693.3
No. Observations:                1000   AIC:                                      7391.
Df Residuals:                     998   BIC:                                      7400.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             9.1817      0.800     11.483      0.000       7.613      10.751
x2            -7.1191      0.797     -8.934      0.000      -8.683      -5.555
==============================================================================
Omnibus:                        3.203   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.202   Jarque-Bera (JB):                2.706
Skew:                          -0.003   Prob(JB):                        0.258
Kurtosis:                       2.745   Cond. No.                         2.65
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""